In [88]:
from sklearn.svm import SVC
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn import svm, datasets
import random
from imblearn.over_sampling import SVMSMOTE, BorderlineSMOTE, KMeansSMOTE
%matplotlib inline

In [57]:
df = pd.read_csv('task-B-Scores.csv')

In [58]:
df.shape

(843, 9)

In [59]:
true_index = []
false_index = []

for i in range(df.shape[0]):
    if df['Labels'][i] == True:
        true_index.append(i)
    else:
        false_index.append(i)

In [60]:
random_true = random.sample(true_index, 32)
random_false = random.sample(false_index, 200)

In [61]:
df1 = df.drop(df.index[random_true + random_false], axis = 0)
df2 = pd.concat([df1, df1[df1['Labels']==True]])

In [62]:
df2.shape

(694, 9)

In [63]:
validation = pd.concat([df.iloc[random_true], df.iloc[random_false]])

In [64]:
df2 = df2.sample(frac=1).reset_index(drop=True)

In [65]:
X = df2.drop(['Labels', 'Score_1', 'Rouge_1','Score_2', 'Rouge_l','Sequence_matcher', 'Unnamed: 0'], axis=1)
y = df2['Labels']

In [66]:
X

,Cosine_Similarity,Res
0,0.000000,0.000000
1,0.103756,3.539823
2,0.337884,16.826923
3,0.127838,4.347826
4,0.168359,4.901961
...,...,...
689,0.055048,2.500000
690,0.030069,1.086957
691,0.120060,3.980100
692,0.043519,1.075269


In [67]:
# df1 = df.drop('Unnamed: 0', axis = 1)
# df1['Score_2'] = df1['Score_2'].fillna(0)

In [68]:
# X_train, X_test, y_train, y_test = train_test_split(df1.drop(['Labels', 'Score_1', 'Rouge_1', 'Score_2', 'Rouge_l','Sequence_matcher'], axis=1), df1['Labels'], test_size=0.15, random_state=0)

In [69]:
sm = KMeansSMOTE(random_state=10)

In [70]:
X_res, y_res = sm.fit_resample(X, y)

In [71]:
X_res.shape

(1058, 2)

In [72]:
clf = SVC(kernel= 'rbf', gamma= 100)

In [73]:
clf.fit(X_res,y_res)

SVC(gamma=100)

In [74]:
pred = clf.predict(validation.drop(['Labels', 'Score_1', 'Rouge_1', 'Score_2', 'Rouge_l','Sequence_matcher', 'Unnamed: 0'], axis =1))

In [75]:
acc = accuracy_score(validation['Labels'], pred)
pre = precision_score(validation['Labels'], pred)
rec = recall_score(validation['Labels'], pred)
print('Accuracy: ', acc)
print('Precision: ', pre)
print('Recall: ', rec)
confusion_matrix(validation['Labels'], pred)

Accuracy:  0.8879310344827587
Precision:  0.65
Recall:  0.40625


array([[193,   7],
       [ 19,  13]])

In [85]:
clf_rf = RandomForestClassifier(max_depth= 100, random_state=10)
clf_rf.fit(X_res, y_res)

RandomForestClassifier(max_depth=100, random_state=10)

In [86]:
predrf = clf_rf.predict(validation.drop(['Labels', 'Score_1', 'Rouge_1', 'Score_2', 'Rouge_l','Sequence_matcher', 'Unnamed: 0'], axis =1))

In [89]:
accrf = accuracy_score(validation['Labels'], predrf)
pre = precision_score(validation['Labels'], predrf)
rec = recall_score(validation['Labels'], predrf)
f1 = f1_score(validation['Labels'], predrf)
print('Accuracy: ', accrf)
print('Precision: ', pre)
print('Recall: ', rec)
print('F1 Accuracy: ', f1)
confusion_matrix(validation['Labels'], predrf)

Accuracy:  0.9137931034482759
Precision:  0.7142857142857143
Recall:  0.625
F1 Accuracy:  0.6666666666666666


array([[192,   8],
       [ 12,  20]])

In [24]:
test_data = pd.read_csv('task-b-test-scores.csv')

In [25]:
test_data.columns

Index(['Unnamed: 0', 'Cosine_Similarity', 'Rouge_1', 'Rouge_l',
       'Sequence_matcher', 'Res', 'Score_1', 'Score_2', 'File_name'],
      dtype='object')

In [26]:
test_prediction = clf_rf.predict(test_data.drop([ 'Score_1', 'Rouge_l', 'Score_2', 'Rouge_1','Sequence_matcher', 'Unnamed: 0', 'File_name'], axis=1))

In [27]:
test_prediction.shape

(972,)

In [28]:
# test_prediction.reshape(972,1)

In [29]:
lables = pd.DataFrame(test_prediction, columns= [ 'Label'])

In [30]:
final_result = pd.concat([test_data, lables], axis = 1)

In [31]:
final_result.columns

Index(['Unnamed: 0', 'Cosine_Similarity', 'Rouge_1', 'Rouge_l',
       'Sequence_matcher', 'Res', 'Score_1', 'Score_2', 'File_name', 'Label'],
      dtype='object')

In [32]:
final_result_lablels = x = final_result.drop(['Cosine_Similarity','Unnamed: 0', 'Rouge_l', 'Rouge_1', 'Sequence_matcher', 'Res', 'Score_1', 'Score_2'], axis = 1)

In [33]:
x[x['Label']==True].shape

(66, 2)

In [34]:
# final_result_lablels.to_csv('Task B_Predictions_EN.tsv', sep= '\t')